# Reading the Dataset

In [35]:
import pandas as pd
import numpy as np

data = pd.read_csv('diabetes.csv')

x = data.drop('Outcome', axis=1)
y = data['Outcome']

x = x.values
y = y.values

y = y.reshape(-1, 1)

# Splitting the data into training and test sets

In [36]:
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.15, random_state=42)

# Defining and Implementing the Neural Network Class

In [41]:
class NeuralNetwork:
    def __init__(self, inputSize, hiddenSize1, hiddenSize2, outputSize):
        self.layer1Weights = np.random.randn(inputSize, hiddenSize1)
        self.layer1Bias = np.zeros((1, hiddenSize1))
        self.layer2Weights = np.random.randn(hiddenSize1, hiddenSize2)
        self.layer2Bias = np.zeros((1, hiddenSize2))
        self.layer3Weights = np.random.randn(hiddenSize2, outputSize)
        self.layer3Bias = np.zeros((1, outputSize))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def deriveSigmoid(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.layer1Input = np.dot(X, self.layer1Weights) + self.layer1Bias
        self.layer1 = self.sigmoid(self.layer1Input)
        self.layer2Input = np.dot(self.layer1, self.layer2Weights) + self.layer2Bias
        self.layer2 = self.sigmoid(self.layer2Input)
        self.outputLayerInput = np.dot(self.layer2, self.layer3Weights) + self.layer3Bias
        self.output = self.sigmoid(self.outputLayerInput)
        return self.output

    def backward(self, X, y, learning_rate):

        error = y - self.output
        dOutput = error * self.deriveSigmoid(self.output)


        errorLayer2 = dOutput.dot(self.layer3Weights.T)
        dLayer2 = errorLayer2 * self.deriveSigmoid(self.layer2)


        errorLayer1 = dLayer2.dot(self.layer2Weights.T)
        dLayer1 = errorLayer1 * self.deriveSigmoid(self.layer1)


        self.layer3Weights += self.layer2.T.dot(dOutput) * learning_rate
        self.layer3Bias += np.sum(dOutput, axis=0, keepdims=True) * learning_rate
        self.layer2Weights += self.layer1.T.dot(dLayer2) * learning_rate
        self.layer2Bias += np.sum(dLayer2, axis=0, keepdims=True) * learning_rate
        self.layer1Weights += X.T.dot(dLayer1) * learning_rate
        self.layer1Bias += np.sum(dLayer1, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs, learningRate):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, learningRate)

# Training and Evaluating with the dataset

In [42]:
inputSize = xTrain.shape[1]
hiddenSize1 = 8
hiddenSize2 = 4
outputSize = 1

#initialzing the neural network
nn = NeuralNetwork(inputSize, hiddenSize1, hiddenSize2, outputSize)

#training the neural network
nn.train(xTrain, yTrain, 30000, 0.01)

#making predictions on the test data
output = nn.forward(xTest)

#converting to binary predictions
predictions = (output > 0.5).astype(int)

#Evaluating the overall accuracy on the test set
accuracy = np.mean(predictions == yTest)
print(f"Accuracy: {accuracy}")

<ipython-input-41-41460ba3fb3c>:11: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Accuracy: 0.7068965517241379
